# Zonal statistics in python using Earth Engine and Google Colab


In this notebook, I will show how to calculate zonal statistics for multiple parameters (mean, median, standard deviation, variance, minimum and maximum) for multiple raster layers simultaneously using the Earth Engine Python API in Google Colab.

The following data will be used for the work:

**Data:**

* Polygons: [Natural Earth (1:110m)](https://www.naturalearthdata.com/downloads/110m-cultural-vectors/) Admin 0 – Countries without large lakes.

* Rasters:

  * Relief - [ETOPO1: Global 1 Arc-Minute Elevation](https://developers.google.com/earth-engine/datasets/catalog/NOAA_NGDC_ETOPO1)
  * Soil temperature - [ERA5-Land Monthly Averaged by Hour of Day - ECMWF Climate Reanalysis](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_MONTHLY_BY_HOUR)
  * Vegetation - [MOD13A3.061 Vegetation Indices Monthly L3 Global 1 km SIN Grid](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13A3)

Откройте сайт [Natural Earth (1:110m)](https://www.naturalearthdata.com/downloads/110m-cultural-vectors/) в новой вкладке и скачайте файл Admin 0 – Countries  without boundary lakes.

Next, execute the code cells in this Сolab notebook one by one.


## Intro

### Initialize environment


In [ ]:
# -------------------------------
# Initializing the Environment (Colab)
# -------------------------------

# Install Required Packages

# Import a helper module from Google Colab for managing output.
# Used, for example, to clean up a cell after installing packages.
from google.colab import output # Only needed in Colab. This module is not available in regular Jupyter.

# Installing Required Libraries.

# Shell commands are supported in Colab, using !. Here, we install GeoPandas and geemap.
!pip install -q geopandas geemap

# Import standard and external libraries that we will use below.
import os
from google.colab import files # Convenient for uploading/downloading files to Colab.

import ee # Google Earth Engine Python API
import geopandas as gpd
import geemap

# Clear cell output after installation/import to make the notebook look cleaner.
output.clear()

# Authorization in GEE

In [ ]:
# -------------------------------
# GEE Authentication and Initialization
# -------------------------------

# ee.Authenticate() starts the interactive authorization flow:
# - In Colab, it will redirect to the Google account login page.
# - You need to log in to your account. This is done once per session.
# NOTE: If you are already logged in in this session, you do not need to call it again.

ee.Authenticate()



# ee.Initialize(...) establishes a connection to Earth Engine.
# The project= parameter specifies the GCP project under which requests (quotas/billing) will be made.
# Must match your GCP project linked to Earth Engine.

# Replace 'MY-GEE-PROJECT' with your actual GEE/GCP project.

ee.Initialize(project='MY-GEE-PROJECT')




# Defining reducers to calculate the required statistics



In [ ]:


# ---------------------------------------------
# Composite reducer for summary statistics
# ---------------------------------------------
# Idea: combine several reducers into one to pass over the data only once.
# sharedInputs=True is an important flag: the same inputs are fed to all reducers,
# which saves compute time and quotas.

reducers = (
    ee.Reducer.mean()      # mean value (output 'mean')
      .combine(ee.Reducer.median(),   sharedInputs=True)   # median ('median')
      .combine(ee.Reducer.stdDev(),   sharedInputs=True)   # standard deviation ('stdDev')
      .combine(ee.Reducer.variance(), sharedInputs=True)   # variance ('variance')
      .combine(ee.Reducer.min(),      sharedInputs=True)   # minimum ('min')
      .combine(ee.Reducer.max(),      sharedInputs=True)   # maximum ('max')
)

# NOTE:
# - Each base reducer has its own output names. When combined, they are concatenated.
#   For multiband images, GEE will add suffixes to band names, e.g., NDVI_mean, NDVI_median, ...



# Import images

In [ ]:
# ---------------------------------------------
# Raster layers: elevation (ETOPO1), NDVI (MODIS), soil temperature (ERA5-Land)
# ---------------------------------------------

# Relief: ETOPO1 bedrock (meters)
elev_dataset = ee.Image('NOAA/NGDC/ETOPO1')
elevation = elev_dataset.select('bedrock')  # "bedrock" layer
# Land mask: keep only >= 0 m (seas/oceans with negative elevation are masked out)
elevation = elevation.updateMask(elevation.gte(0)).rename('elevation')

# Vegetation: MODIS MOD13A3 for 2024, annual mean
# NDVI has a scale factor of 0.0001 and a valid range of approximately [-2000..10000] before scaling.
vi_dataset = (
    ee.ImageCollection('MODIS/061/MOD13A3')
    .filter(ee.Filter.date('2024-01-01', '2024-12-31'))
    .mean()
)

ndvi_raw = vi_dataset.select('NDVI')
ndvi = (
    ndvi_raw
    .multiply(0.0001)                    # convert to the [-1..1] range
    .updateMask(ndvi_raw.neq(-3000))     # remove fill values (typical for MODIS)
    .rename('ndvi')
)

# Climate: ERA5-Land MONTHLY_BY_HOUR for 2024, annual mean
# Units of 'soil_temperature_level_1' are Kelvin. Convert to °C.

ECMWF_dataset = (
    ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_BY_HOUR')
    .filter(ee.Filter.date('2024-01-01', '2024-12-31'))
    .mean()
)

temperature_k = ECMWF_dataset.select('soil_temperature_level_1')
temperature_c = temperature_k.subtract(273.15).rename('temperature_c')

# ---------------------------------------------
# Merge layers into a single image
# ---------------------------------------------
# Band order: elevation (m), NDVI (unitless), soil temperature (°C).
# Important: the layers have different native projections/resolutions; when using reduceRegion/export
# specify an explicit scale (or crs/crsTransform) to avoid unwanted resampling.

image_to_process = elevation.addBands([ndvi, temperature_c])


# Process vector file

You can upload a vector file to session storage using the corresponding button in the interface.

Then pass the path to the uploaded file to the `polygons_filename` variable.

In [ ]:
# ---------------------------------------------
# Uploading a vector file in Colab
# ---------------------------------------------
# After running, a "Choose files" dialog will appear.
# Important: a shapefile consists of multiple files (.shp, .shx, .dbf, .prj, etc.).
# It's more convenient to upload a single ZIP containing the set, or use GeoJSON/GPKG/KML formats.

uploaded = files.upload()  # dict: {filename: bytes}

# Basic (current) approach: assume a single file and just take its name.
# If multiple are uploaded, the first in the dictionary order is taken.

polygons_filename = list(uploaded.keys())[0]

print(f"File '{polygons_filename}' loaded successfully.")

Saving ne_110m_admin_0_countries_lakes.zip to ne_110m_admin_0_countries_lakes.zip
File 'ne_110m_admin_0_countries_lakes.zip' loaded successfully.


In [ ]:
# Get the base filename without the extension.
# os.path.splitext will cut off only the last extension (e.g., '.zip' or '.shp').
# Example: 'my.layer.v1.geojson' -> ('my.layer.v1', '.geojson')

polygons_basename = os.path.splitext(polygons_filename)[0]
print(f"polygons_basename: {polygons_basename}")


polygons_basename: ne_110m_admin_0_countries_lakes


In [ ]:
# Reading a vector file with geopandas

polygons_gdf = gpd.read_file(polygons_filename)

# Checking the coordinate reference system (CRS) of the vector layer

if polygons_gdf.crs is None:
    print("⚠️ The layer is missing a CRS. If you know the original projection, set it manually, otherwise the results may be incorrect.")
else:
    if polygons_gdf.crs.to_string().upper() != "EPSG:4326":
        polygons_gdf = polygons_gdf.to_crs(epsg=4326)
        print("Projection redefined to WGS84 (EPSG:4326).")

# Converting GeoDataFrame to ee.FeatureCollection
# This is a key step to pass the data into GEE

polygons_ee = geemap.geopandas_to_ee(polygons_gdf)


In [ ]:
# ---------------------------------------------
# Zonal statistics over polygons
# ---------------------------------------------
# reduceRegions applies a composite reducer to each feature in a FeatureCollection.
# - Preserves the original polygon attributes.
# - Adds fields with statistics for each image band.
#   Example names: elevation_mean, ndvi_median, temperature_c_stdDev, ...
# WARNING: different layers have different native resolutions; with the `scale` parameter
# we set a unified sampling resolution (here 5000 m, which is coarser than MODIS (~1 km)
# and ETOPO1 (~1.8 km), but closer to ERA5-Land (~9 km)). Choose `scale` deliberately for your task.

stats_results_ee = image_to_process.reduceRegions(
    collection=polygons_ee,  # polygons (ee.FeatureCollection)
    reducer=reducers,        # previously assembled composite reducer
    scale=5000               # pixel size for sampling (m)
)


In [ ]:
%%time
# ---------------------------------------------
# Converting ee.FeatureCollection -> GeoDataFrame
# ---------------------------------------------
# geemap.ee_to_gdf pulls ALL features to the client (into the notebook's memory).
# This is convenient for small/medium datasets, but can be heavy for large collections.

stats_results_gdf = geemap.ee_to_gdf(stats_results_ee)



CPU times: user 2.46 s, sys: 92.6 ms, total: 2.55 s
Wall time: 8.78 s


In [ ]:
# Quick overview of columns and types
stats_results_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 177 entries, 0 to 176
Columns: 187 entries, geometry to temperature_c_variance
dtypes: float64(22), geometry(1), int64(27), object(137)
memory usage: 258.7+ KB


In [ ]:
# View the first four lines of the result.

stats_results_gdf.head(4)

,geometry,ABBREV,ABBREV_LEN,ADM0_A3,ADM0_A3_AR,ADM0_A3_BD,ADM0_A3_BR,ADM0_A3_CN,ADM0_A3_DE,ADM0_A3_EG,...,ndvi_min,ndvi_stdDev,ndvi_variance,scalerank,temperature_c_max,temperature_c_mean,temperature_c_median,temperature_c_min,temperature_c_stdDev,temperature_c_variance
0,"MULTIPOLYGON (((-180 -16.55522, -179.91737 -16...",Fiji,4,FJI,FJI,FJI,FJI,FJI,FJI,FJI,...,-0.044917,0.109489,0.011988,1,28.706537,24.806245,24.964065,21.944154,1.069099,1.142973
1,"POLYGON ((29.60009 -4.89639, 29.79196 -5.04088...",Tanz.,5,TZA,TZA,TZA,TZA,TZA,TZA,TZA,...,-0.171250,0.128486,0.016509,1,34.006701,24.587345,24.686355,15.017911,2.111786,4.459639
2,"POLYGON ((-17.06342 20.99975, -16.95443 21.166...",W. Sah.,7,SAH,SAH,SAH,SAH,SAH,SAH,SAH,...,0.002550,0.015187,0.000231,1,31.752693,28.053523,27.733953,24.567926,1.574773,2.479910
3,"MULTIPOLYGON (((-140.99778 60.30639, -140.7514...",Can.,4,CAN,CAN,CAN,CAN,CAN,CAN,CAN,...,-0.199400,0.190048,0.036118,1,14.208544,1.321117,3.125350,-29.567904,7.079515,50.119535


In [ ]:
# Preview of the first feature

stats_results_ee.first().getInfo()


{'type': 'Feature',
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[180, -16.067132663642447],
     [179.4135093629971, -16.379054277547404],
     [179.0966093629971, -16.433984277547403],
     [178.59683859511713, -16.63915],
     [178.72505936299711, -17.01204167436804],
     [179.36414266196414, -16.801354076946883],
     [180, -16.555216566639196],
     [180, -16.067132663642447]]],
   [[[178.12557, -17.50481],
     [177.67087, -17.38114],
     [177.28504, -17.72465],
     [177.38146, -18.16432],
     [177.93266, -18.28799],
     [178.55271, -18.15059],
     [178.71806, -17.62846],
     [178.3736, -17.33992],
     [178.12557, -17.50481]]],
   [[[-179.79332010904864, -16.020882256741224],
     [-180, -16.067132663642447],
     [-180, -16.555216566639196],
     [-179.9173693847653, -16.501783135649397],
     [-179.79332010904864, -16.020882256741224]]]]},
 'id': '0',
 'properties': {'ABBREV': 'Fiji',
  'ABBREV_LEN': 4,
  'ADM0_A3': 'FJI',
  'ADM0_A3_AR': 'FJI',
  'ADM0_A3_

In [ ]:
# ---------------------------------------------
# Reordering and cleaning columns
# ---------------------------------------------

# Define a new column order. Remove extras and keep the necessary fields, placing them in the desired order.

new_order = [
    'geometry', 'ABBREV', 'NAME', 'CONTINENT', 'ECONOMY', 'ISO_A2', 'ISO_A3', 'ISO_N3',
    'elevation_mean', 'elevation_median', 'elevation_min', 'elevation_max', 'elevation_stdDev', 'elevation_variance',
    'ndvi_mean', 'ndvi_median', 'ndvi_min', 'ndvi_max', 'ndvi_stdDev', 'ndvi_variance',
    'temperature_c_mean', 'temperature_c_median', 'temperature_c_min', 'temperature_c_max', 'temperature_c_stdDev', 'temperature_c_variance'
]

# Apply the new order
stats_results_gdf = stats_results_gdf[new_order]

# View the first few rows of the result
stats_results_gdf.head()


,geometry,ABBREV,NAME,CONTINENT,ECONOMY,ISO_A2,ISO_A3,ISO_N3,elevation_mean,elevation_median,...,ndvi_min,ndvi_max,ndvi_stdDev,ndvi_variance,temperature_c_mean,temperature_c_median,temperature_c_min,temperature_c_max,temperature_c_stdDev,temperature_c_variance
0,"MULTIPOLYGON (((-180 -16.55522, -179.91737 -16...",Fiji,Fiji,Oceania,6. Developing region,FJ,FJI,242,223.468646,179.525346,...,-0.044917,0.892175,0.109489,0.011988,24.806245,24.964065,21.944154,28.706537,1.069099,1.142973
1,"POLYGON ((29.60009 -4.89639, 29.79196 -5.04088...",Tanz.,Tanzania,Africa,7. Least developed region,TZ,TZA,834,1024.537119,1136.110604,...,-0.171250,0.890683,0.128486,0.016509,24.587345,24.686355,15.017911,34.006701,2.111786,4.459639
2,"POLYGON ((-17.06342 20.99975, -16.95443 21.166...",W. Sah.,W. Sahara,Africa,7. Least developed region,EH,ESH,732,317.995099,301.408844,...,0.002550,0.192142,0.015187,0.000231,28.053523,27.733953,24.567926,31.752693,1.574773,2.479910
3,"MULTIPOLYGON (((-140.99778 60.30639, -140.7514...",Can.,Canada,North America,1. Developed region: G7,CA,CAN,124,456.829791,334.896322,...,-0.199400,0.965500,0.190048,0.036118,1.321117,3.125350,-29.567904,14.208544,7.079515,50.119535
4,"MULTIPOLYGON (((-171.79111 63.40585, -171.6719...",U.S.A.,United States of America,North America,1. Developed region: G7,US,USA,840,711.817240,431.114519,...,-0.188500,0.888408,0.180770,0.032678,11.116145,11.627143,-18.517672,28.737672,7.358007,54.140264


The stats_results_gdf object is a GeoDataFrame and you can convert it to any vector format supported by the GeoPandas library.

In [ ]:
# ---------------------------------------------
# Saving results to GPKG and CSV
# ---------------------------------------------

layer_name = "zonal_statistics"  # layer name in GPKG
output_gpkg_filename = f'{polygons_basename}_zonal_statistics.gpkg'
output_csv_filename = f'{polygons_basename}_zonal_statistics.csv'

# Save GeoPackage (geometry + attributes)
# driver='GPKG' is a required parameter. layer= sets the layer name inside the file, optional parameter.

stats_results_gdf.to_file(output_gpkg_filename, driver='GPKG', layer=layer_name)

# Save CSV (attributes only)
# errors='ignore' prevents errors if the geometry column is missing.

stats_results_gdf.drop(columns="geometry", errors="ignore").to_csv(
    output_csv_filename,
    index=False,
    encoding="utf-8"  # for Cyrillic
    # float_format="%.6f"  # enable if you want a fixed number of decimal places
    )

print(f"Saved:\n- {output_gpkg_filename} (layer: {layer_name})\n- {output_csv_filename}")


Saved:
- ne_110m_admin_0_countries_lakes_zonal_statistics.gpkg (layer: zonal_statistics)
- ne_110m_admin_0_countries_lakes_zonal_statistics.csv


You can download the finished files by selecting them in your browser's file system.

In [ ]:
# You can also use the code for automatic file download (optional)

files.download(output_gpkg_filename)
files.download(output_csv_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ---------------------------------------------
# Exporting the resulting table to Google Drive
# ---------------------------------------------

# You can also export the stats_results_ee object, which is an ee.FeatureCollection, to Google Drive directly without transforming it into a GeoDataFrame.
# Choose a format: 'GeoJSON' (with geometry), 'CSV' (attributes only), 'SHP' (has attribute name length limits), 'KML', 'KMZ'
# For large and complex geometries, GeoJSON is usually more convenient

result_filename = f'{polygons_basename}_zonal_statistics'
export_folder = 'GEE_Exports' # Folder on your Google Drive
file_format = 'GeoJSON'

task = ee.batch.Export.table.toDrive(
    collection=stats_results_ee,           # ee.FeatureCollection
    description=result_filename,           # task name and filename without extension
    folder=export_folder,                  # folder in Google Drive
    fileFormat=file_format                 # file format
)

# Start the export task
task.start()

print('Export task started. Check the Earth Engine Tasks tab for progress:')
print(task.status())


Export task started. Check the Earth Engine Tasks tab for progress:
{'state': 'READY', 'description': 'ne_110m_admin_0_countries_lakes_zonal_statistics', 'priority': 100, 'creation_timestamp_ms': 1759564433647, 'update_timestamp_ms': 1759564433647, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'Z27DDSV6KEMJOXLS3UMLMHNM', 'name': 'projects/gee-antonbiatov/operations/Z27DDSV6KEMJOXLS3UMLMHNM'}


Exported GeoJSON, GeoPackage, and CSV files can be used in various analytical and GIS tools.

Python (GeoPandas, Matplotlib, etc.) R(sf, ggplot2) QGIS, ArcGIS, and other GIS systems Web mapping software

You can also integrate this analysis method into your Python workflow.

# Limitations of the method

The described method has a serious limitation.

When processing large vector files, you'll quickly encounter the error:

`Request payload size exceeds the limit: 10485760 bytes`

This means the data transferred in the Earth Engine API request is larger than 10 MB.

In the next article, we'll look at several ways to circumvent this limitation.

---

See also the article about [Zonal Statistics in the Google Earth Engine web interface >>>](https://medium.com/@anton.biatov/how-to-combine-topography-and-vegetation-a-practical-guide-to-zonal-statistics-in-google-earth-d8167615937b)





# Preview Data on maps

The next step is optional, since you already have the data.
In this step, we'll simply look at the resulting statistics on the map.


In [ ]:
# Rasterize polygons with calculation results for quick preview on the map

elev_mean_img = (
    stats_results_ee
    .reduceToImage(properties=['elevation_mean'], reducer=ee.Reducer.first())
    .rename('elevation')
)

elev_stdDev_img = (
    stats_results_ee
    .reduceToImage(properties=['elevation_stdDev'], reducer=ee.Reducer.first())
    .rename('elevation')
)

elev_variance_img = (
    stats_results_ee
    .reduceToImage(properties=['elevation_variance'], reducer=ee.Reducer.first())
    .rename('elevation')
)

ndvi_mean_img = (
    stats_results_ee
    .reduceToImage(properties=['ndvi_mean'], reducer=ee.Reducer.first())
    .rename('ndvi')
)

ndvi_stdDev_img = (
    stats_results_ee
    .reduceToImage(properties=['ndvi_stdDev'], reducer=ee.Reducer.first())
    .rename('ndvi')
)

ndvi_variance_img = (
    stats_results_ee
    .reduceToImage(properties=['ndvi_variance'], reducer=ee.Reducer.first())
    .rename('ndvi')
)

temperature_mean_img = (
    stats_results_ee
    .reduceToImage(properties=['temperature_c_mean'], reducer=ee.Reducer.first())
    .rename('temperature')
)

temperature_stdDev_img = (
    stats_results_ee
    .reduceToImage(properties=['temperature_c_stdDev'], reducer=ee.Reducer.first())
    .rename('temperature')
)

temperature_variance_img = (
    stats_results_ee
    .reduceToImage(properties=['temperature_c_variance'], reducer=ee.Reducer.first())
    .rename('temperature')
)

In [ ]:
# Let's set up palettes for displaying layers

palette_elev = [
    "006837","1a9850","66bd63","a6d96a","d9ef8b",
    "ffffbf","fee08b","fdae61","f46d43","d73027","a50026"
]

palette_ndvi = [
    'ffffff','fcd163','99b718','66a000', '3e8601',
    '207401','056201','004c00','011301'
    ]

palette_temperature = [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]

palette_std = [
    "5e4fa2","3288bd","66c2a5","abdda4","e6f598",
    "fee08b","fdae61","f46d43","d53e4f","9e0142","5e4fa2"
]



## Displaying the obtained statistics on the map


In [ ]:
# Display terrain statistics


# Visualization parameters for different layers
vis_elev = dict(min=50, max=4000, palette=palette_elev)
vis_elev_mean = dict(min=50, max=2000, palette=palette_elev)
vis_elev_std = dict(min=0, max=1500, palette=palette_std)
vis_elev_variance = dict(min=0, max=3000000, palette=palette_std)

# List of images to display (order corresponds to vis_params and labels)
images = [
    elevation,
    elev_mean_img,
    elev_stdDev_img,
    elev_variance_img
]

# Corresponding visualization parameters for each image in the list above
vis_params = [
    vis_elev,
    vis_elev_mean,
    vis_elev_std,
    vis_elev_variance
]

# Layer labels (shown in each linked map's title)
labels = [
    "Elevation",
    "Elev. Mean",
    "Elev. Standard deviation",
    "Elev. Variance"
]

# Create 4 linked maps (panning/zoom are synchronized)
geemap.linked_maps(
    rows=1,                     # number of rows (1 row)
    cols=4,                     # number of columns (4 maps side by side)
    height="700px",             # widget height
    center=[10, -80],           # map center [lat, lon]
    zoom=2,                     # initial zoom level
    ee_objects=images,          # list of ee.Image objects to display
    vis_params=vis_params,      # list of visualization parameters (must match length of images)
    labels=labels,              # map labels (length must also match)
    label_position="topright",  # label position on each map
)


GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [ ]:
# Display vegetation statistics

vis_ndvi = dict(min=0, max=0.8, palette=palette_ndvi)
vis_ndvi_mean = dict(min=0, max=0.8, palette=palette_ndvi)
vis_ndvi_std = dict(min=0, max=0.25, palette=palette_std)
vis_ndvi_variance = dict(min=0, max=0.05, palette=palette_std)


images = [
    ndvi,
    ndvi_mean_img,
    ndvi_stdDev_img,
    ndvi_variance_img,
]

vis_params = [
    vis_ndvi,
    vis_ndvi_mean,
    vis_ndvi_std,
    vis_ndvi_variance,
]

labels = [
    "NDVI",
    "NDVI Mean",
    "NDVI Standard deviation",
    "NDVI Variance",
]

geemap.linked_maps(
    rows=1,
    cols=4,
    height="700px",
    center=[10, -80],
    zoom=2,
    ee_objects=images,
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [ ]:
# Display temperature statistics

vis_temperature = dict(min=0, max=35, palette=palette_temperature)
vis_temperature_mean = dict(min=0, max=35, palette=palette_temperature)
vis_temperature_std = dict(min=0, max=9, palette=palette_std)
vis_temperature_variance = dict(min=0, max=70, palette=palette_std)

images = [
    temperature_c,
    temperature_mean_img,
    temperature_stdDev_img,
    temperature_variance_img
]

vis_params = [
    vis_temperature,
    vis_temperature_mean,
    vis_temperature_std,
    vis_temperature_variance
]

labels = [
    "Temperature",
    "Temperature Mean",
    "Temperature Standard deviation",
    "Temperature Variance"
]

geemap.linked_maps(
    rows=1,
    cols=4,
    height="700px",
    center=[10, -80],
    zoom=2,
    ee_objects=images,
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

Ready to try? Copy of this Colab notebook and adapt it to your needs and adapt it to your needs.

If this article was helpful, follow my page and share your results or questions in the comments. Together we can tackle global challenges with data and technology!

[LinkedIn](https://www.linkedin.com/in/anton-biatov/)

[X.com (Twitter)](https://x.com/GisNaturalist)

[Instagram](https://www.instagram.com/anton_biatov/)

[Medium](https://medium.com/@anton.biatov)

[YouTube](https://www.youtube.com/@antonbiatov9203)

[Facebook](https://www.facebook.com/gisnaturalist)